# Shoprite Coupon Clipper

This notebook contains the initial development of the Shoprite Coupon Clipper. 

## Initial Website Access

Initiate a valid connection to [shoprite.com](shoprite.com)


In [1]:
# import libraries
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
from playwright_stealth import stealth_async
import os
from dotenv import load_dotenv


In [4]:
nest_asyncio.apply()

# Load credentials from .env file
load_dotenv()
username = os.getenv("SHOPRITE_USERNAME")
password = os.getenv("SHOPRITE_PASSWORD")
home_page = "https://www.shoprite.com"

# Ensure username and password are set
if not username or not password:
    raise ValueError("Missing SHOPRITE_USERNAME or SHOPRITE_PASSWORD in environment variables")

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        await stealth_async(page)
        
        # Open Shoprite home page
        await page.goto(home_page, timeout=60000)
        print("Opened Shoprite home page")
        
        # Reject all cookies
        cookie_button = await page.query_selector("#onetrust-reject-all-handler")
        if cookie_button:
            await cookie_button.click()
            print("Rejected cookies.")
        else:
            print("Cookie popup not found, continuing...")
        
        print("Navigating to login page")
        await page.pause()
        
        # Navigate to login page
        await page.wait_for_selector("#AccountHeaderButton", timeout=10000)  # Wait for button to load
        await page.click("#AccountHeaderButton")
        print("Navigated to login page.")

        # Ensure the login form has loaded
        await page.wait_for_selector("input[type='email']", timeout=10000)

        # Fill in login details
        await page.fill("input[type='email']", username)
        await page.fill("input[type='password']", password)

        # Click login button
        await page.click("button[type='submit']")

        # Wait for the page to load after login
        await page.wait_for_load_state("networkidle")

        print("Login successful!")

        await asyncio.Event().wait()

asyncio.get_event_loop().run_until_complete(run())

Opened Shoprite home page
Cookie popup not found, continuing...
Navigating to login page
Navigated to login page.


TimeoutError: Page.wait_for_selector: Timeout 10000ms exceeded.
Call log:
  - waiting for locator("input[type='email']") to be visible
